Find the markdown blocks that say interaction required! The notebook should take care of the rest!

# Import libs

In [1]:
import sys
import os
sys.path.append('..')
from eflow.foundation import DataPipeline,DataFrameTypes
from eflow.model_analysis import RegressionAnalysis
from eflow.utils.modeling_utils import optimize_model_grid
from eflow.utils.eflow_utils import get_type_holder_from_pipeline, remove_unconnected_pipeline_segments
from eflow.utils.pandas_utils import data_types_table

import pandas as pd
import numpy as np
import scikitplot as skplt
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
import copy
import pickle
from IPython.display import clear_output

In [2]:
# # Additional add ons
# !pip install pandasgui
# !pip install pivottablejs
# clear_output()

In [3]:
%matplotlib notebook
%matplotlib inline

## Declare Project Variables

### Interaction required

In [4]:
dataset_path = "Datasets/titanic_train.csv"

# -----
dataset_name = "Titanic Data"
pipeline_name = "Titanic Pipeline"

# -----


# -----
notebook_mode = True

## Clean out segment space

In [5]:
remove_unconnected_pipeline_segments()

# Import dataset

In [6]:
df = pd.read_csv(dataset_path)
shape_df = pd.DataFrame.from_dict({'Rows': [df.shape[0]],
                                   'Columns': [df.shape[1]]})
display(shape_df)
display(df.head(30))

,Rows,Columns
0,891,12


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [7]:
data_types_table(df)

,Data Types
Features,
Age,float64
Fare,float64
PassengerId,int64
Survived,int64
Pclass,int64
SibSp,int64
Parch,int64
Name,object
Sex,object


# Loading and init df_features

In [8]:
# Option: 1
# df_features = get_type_holder_from_pipeline(pipeline_name)

In [9]:
# Option: 2
df_features = DataFrameTypes()
df_features.init_on_json_file(os.getcwd() + f"/eflow Data/{dataset_name}/df_features.json")

In [10]:
df_features.display_features(display_dataframes=True,
                             notebook_mode=notebook_mode)

,Data Types
Features,
Cabin,string
Embarked,string
Pclass,string
Sex,string
Survived,bool
SibSp,integer
Parch,integer
Age,float
Fare,float


# Any extra processing before eflow DataPipeline

# Setup pipeline structure

### Interaction Required

In [11]:
main_pipe = DataPipeline(pipeline_name,
                         df,
                         df_features)

The file 'root_pipeline.json' exist!
Now configuring object with proper pipeline segments...
Removing the feature: "Ticket"
Removing the feature: "Name"
Removing the feature: "PassengerId"


In [12]:
main_pipe.perform_pipeline(df,
                           df_features)

Ignore feature:  Fare


In [13]:
df

,Survived,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3,Embarked_Cherbourg,Embarked_Queenstown,Embarked_Southampton
0,0,22.000000,1,0,7.2500,False,True,False,False,True,False,False,True
1,1,38.000000,1,0,71.2833,True,False,True,False,False,True,False,False
2,1,26.000000,0,0,7.9250,True,False,False,False,True,False,False,True
3,1,35.000000,1,0,53.1000,True,False,True,False,False,False,False,True
4,0,35.000000,0,0,8.0500,False,True,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,27.000000,0,0,13.0000,False,True,False,True,False,False,False,True
887,1,19.000000,0,0,30.0000,True,False,True,False,False,False,False,True
888,0,29.444268,1,2,23.4500,True,False,False,False,True,False,False,True
889,1,26.000000,0,0,30.0000,False,True,True,False,False,True,False,False


# Seperate out data into train and test sets

In [14]:
X = df.drop(columns=df_features.target_feature()).values
y = df[df_features.target_feature()].values

In [15]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=517,
)

In [16]:
feature_order = list(df.columns)

In [17]:
del df

# Fit Models and view results

In [18]:
# Find best parameters for model
param_grid = {
    "max_depth": list(range(1, 10)),
    'criterion': ["mse", "friedman_mse", "mae"]
}

model, best_params = optimize_model_grid(
    model=DecisionTreeRegressor(),
    X_train=X_train,
    y_train=y_train,
    param_grid=param_grid,
    scoring="r2"
)

Tuned Parameters: {'criterion': 'mse', 'max_depth': 3}
Best score on trained data was 0.358510


/Users/ericcacciavillani/anaconda3/envs/eflow/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [19]:
model.fit(X,y)

DecisionTreeRegressor(criterion='mse', max_depth=3, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

In [20]:
model_name = repr(model).split("(")[0]

In [21]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error, mean_squared_error, mean_squared_log_error, median_absolute_error


In [22]:
model_analysis = RegressionAnalysis(dataset_name=dataset_name,
                                    model=model,
                                    model_name=model_name,
                                    feature_order=feature_order,
                                    target_feature=df_features.target_feature(),
                                    pred_funcs_dict={"Predict function":model.predict},
                                    notebook_mode=notebook_mode,
                                    df_features=df_features)

In [23]:
model_analysis.perform_analysis(X=X_train,
                                y=y_train,
                                dataset_name="Train Data",
                                regression_error_analysis=True,
                                regression_correct_analysis=True,
                                display_visuals=False)



------------------------------Train Data------------------------------


**********Generating graphs for when the model predicted incorrectly**********



**********Generating graphs for when the model predicted correctly**********



In [24]:
for pred_name in model_analysis.get_predictions_names():
    for mse_score in [.2]:
        for gt_lt in ["greater than", "less than"]:
            print(f"Prediction name: {pred_name} with mse_score {gt_lt} {mse_score}")
            infile = open(model_analysis.folder_path + "Train Data" + f"/{pred_name}/MSE score {gt_lt} {mse_score}/Train Data/_Extras/Statistics/Stat methods of features dataframes.pkl",'rb')
            stat_methods_dict = pickle.load(infile)
            infile.close()

            for stats_method in stat_methods_dict.keys():
                print(stats_method)
                display(stat_methods_dict[stats_method].round(6))
                all_feature_relationship = set()
                for feature_relationship in stat_methods_dict[stats_method][:10].index.to_list():
                    for feature in feature_relationship.split(" compared to "):
                        all_feature_relationship.add(feature)
                print(all_feature_relationship)
                print("-----" * 12 + "\n\n")

            del stat_methods_dict

Prediction name: Predict function with mse_score greater than 0.2
Kolmogorov-Smirnov statistic


,mean,std,var
Survived compared to Sex_female,0.000006,0.000011,0.000000
Survived compared to Sex_male,0.000006,0.000011,0.000000
Survived compared to Fare,0.026942,0.043627,0.001903
Survived compared to Pclass_1,0.102080,0.142833,0.020401
Survived compared to Pclass_3,0.145029,0.189340,0.035850
Survived compared to Age,0.625198,0.369362,0.136428
Survived compared to SibSp,0.954196,0.075957,0.005769
Survived compared to Parch,0.986217,0.023639,0.000559
Survived compared to Embarked_Cherbourg,0.986664,0.022881,0.000524
Survived compared to Embarked_Queenstown,0.993717,0.010842,0.000118


{'SibSp', 'Survived', 'Embarked_Queenstown', 'Pclass_3', 'Fare', 'Pclass_1', 'Sex_female', 'Embarked_Cherbourg', 'Age', 'Parch', 'Sex_male'}
------------------------------------------------------------


Prediction name: Predict function with mse_score less than 0.2
Kolmogorov-Smirnov statistic


,mean,std,var
Survived compared to Sex_female,0.000000,0.000000,0.000000
Survived compared to Sex_male,0.000000,0.000000,0.000000
Survived compared to Pclass_3,0.000930,0.001610,0.000003
Survived compared to Fare,0.002309,0.003998,0.000016
Survived compared to Pclass_1,0.040521,0.068041,0.004630
Survived compared to Embarked_Southampton,0.066127,0.107954,0.011654
Survived compared to Parch,0.085332,0.136143,0.018535
Survived compared to Embarked_Cherbourg,0.122326,0.186232,0.034682
Survived compared to Age,0.350886,0.380564,0.144829
Survived compared to Pclass_2,0.380435,0.391782,0.153493


{'Survived', 'Pclass_3', 'Fare', 'Pclass_2', 'Pclass_1', 'Sex_female', 'Embarked_Southampton', 'Embarked_Cherbourg', 'Age', 'Parch', 'Sex_male'}
------------------------------------------------------------




In [25]:
model_analysis.perform_analysis(X=X_test,
                                y=y_test,
                                dataset_name="Test Data",
                                regression_error_analysis=True,
                                regression_correct_analysis=True,
                                display_visuals=False)



------------------------------Test Data------------------------------


**********Generating graphs for when the model predicted incorrectly**********



**********Generating graphs for when the model predicted correctly**********



In [26]:
for pred_name in model_analysis.get_predictions_names():
    for mse_score in [.2]:
        for gt_lt in ["greater than", "less than"]:
            print(f"Prediction name: {pred_name} with mse_score {gt_lt} {mse_score}")
            infile = open(model_analysis.folder_path + "Test Data" + f"/{pred_name}/MSE score {gt_lt} {mse_score}/Test Data/_Extras/Statistics/Stat methods of features dataframes.pkl",'rb')
            stat_methods_dict = pickle.load(infile)
            infile.close()

            for stats_method in stat_methods_dict.keys():
                print(stats_method)
                display(stat_methods_dict[stats_method].round(6))
                all_feature_relationship = set()
                for feature_relationship in stat_methods_dict[stats_method][:10].index.to_list():
                    for feature in feature_relationship.split(" compared to "):
                        all_feature_relationship.add(feature)
                print(all_feature_relationship)
                print("-----" * 12 + "\n\n")

            del stat_methods_dict

Prediction name: Predict function with mse_score greater than 0.2
Kolmogorov-Smirnov statistic


,mean,std,var
Survived compared to Sex_female,0.010325,0.017804,0.000317
Survived compared to Sex_male,0.010325,0.017804,0.000317
Survived compared to Pclass_3,0.356357,0.374038,0.139905
Survived compared to Fare,0.362145,0.375989,0.141368
Survived compared to Pclass_1,0.506797,0.395323,0.156280
Survived compared to Age,0.638085,0.363224,0.131931
Survived compared to Embarked_Cherbourg,0.937017,0.097982,0.009600
Survived compared to Embarked_Queenstown,0.989533,0.017826,0.000318
Survived compared to Parch,0.991321,0.014821,0.000220
Survived compared to Pclass_2,1.000000,0.000001,0.000000


{'Survived', 'Embarked_Queenstown', 'Pclass_3', 'Fare', 'Pclass_2', 'Pclass_1', 'Sex_female', 'Embarked_Cherbourg', 'Age', 'Parch', 'Sex_male'}
------------------------------------------------------------


Prediction name: Predict function with mse_score less than 0.2
Kolmogorov-Smirnov statistic


,mean,std,var
Survived compared to Sex_female,0.000000,0.000001,0.000000
Survived compared to Sex_male,0.000000,0.000001,0.000000
Survived compared to Fare,0.071433,0.121074,0.014659
Survived compared to Pclass_3,0.100044,0.166408,0.027692
Survived compared to SibSp,0.287270,0.381035,0.145188
Survived compared to Pclass_1,0.430584,0.430416,0.185258
Survived compared to Parch,0.497403,0.422391,0.178414
Survived compared to Age,0.607985,0.378682,0.143400
Survived compared to Pclass_2,0.642950,0.359107,0.128958
Survived compared to Embarked_Southampton,1.000000,0.000000,0.000000


{'SibSp', 'Survived', 'Pclass_3', 'Fare', 'Pclass_2', 'Pclass_1', 'Sex_female', 'Embarked_Southampton', 'Age', 'Parch', 'Sex_male'}
------------------------------------------------------------


